In [ ]:
from google.colab import drive
import os
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag

In [ ]:
drive.mount('/content/drive')
datadir = "/content/drive/My Drive/CS546Data/"

Mounted at /content/drive


In [ ]:
import re

def parse_entities_from_file(file_path):
    entities = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Skip lines that don't contain entity definitions
            if line.startswith("<") or line.strip() == '':
                continue
            # Extract entity, ignoring hierarchy indentation
            entity = line.strip().split('\t')[-1]
            entities.append(entity)
    return entities

paths_to_ontology_files = [
    # datadir + 'Ontology/hierarchy.txt',
    datadir + 'Ontology/listEntities.txt'
    # datadir + 'Ontology/listEvents.txt',
    # datadir + 'Ontology/listPredicates.txt',
    # datadir + 'Ontology/listFacts.txt'
]
all_entities = set()

for file_path in paths_to_ontology_files:
    entities_in_file = parse_entities_from_file(file_path)
    all_entities.update(entities_in_file)

# Convert the set to a list and sort it
entities_list = sorted(list(all_entities))

# Write the entities to a text file
with open('entities_list.txt', 'w', encoding='utf-8') as f:
    for entity in entities_list:
        f.write(f"{entity}\n")

In [ ]:
def load_entities(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        entities = [line.strip() for line in file.readlines()]
    return entities

entities_list = load_entities('./entities_list.txt')

In [ ]:
def tag_entities_in_text(text, entities):
    for entity in entities:
        # Create a pattern that matches the entity
        pattern = re.compile(re.escape(entity), re.IGNORECASE)
        # Replace with the tagged version
        text = pattern.sub(f'[ENTITY]{entity}[/ENTITY]', text)
    return text

In [ ]:
def read_and_tag_data(file_path, entities):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    for key, value in data.items():
        tagged_text = tag_entities_in_text(value['description'], entities)
        data[key]['description'] = tagged_text

    return data

tagged_data = read_and_tag_data('path_to_your_minecraft_data.json', entities_list)

In [ ]:
def tag_entities(text, entities):
    tagged_text = text.lower()  # Convert text to lowercase for matching
    for entity in entities:
        # Convert entity to lowercase for matching
        lower_entity = entity.lower()
        # Escape special characters in entity names for regex
        escaped_entity = re.escape(lower_entity)
        # Replace entity with tagged version, use word boundaries to match whole words
        tagged_text = re.sub(rf'\b{escaped_entity}\b', f'[ENTITY]{entity}[/ENTITY]', tagged_text)
    return tagged_text

def read_files_to_json(folder_paths, entity_list):
    data_dict = {}

    if os.path.exists('/content/minecraft_data.json'):
        with open('/content/minecraft_data.json', 'r', encoding='utf-8') as json_file:
            data_dict = json.load(json_file)

    for folder_path in folder_paths:
        for filename in os.listdir(folder_path):
            if filename.endswith(".txt"):
                topic = filename.replace(".txt", "")
                with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                    content = file.read()
                    # Tag the entities in the content
                    content_with_entities = tag_entities_in_text(content, entity_list)

                data_dict[topic] = {
                    'description': content_with_entities  # Store the content with entities tagged
                }

    data_json = json.dumps(data_dict, indent=4)

    with open('/content/minecraft_data.json', 'w', encoding='utf-8') as json_file:
        json_file.write(data_json)

    return data_dict

# Now call the function with the entity list
folder_paths = [
    datadir + 'KnowledgeDatabase/GamepediaTxt/Blocks',
    datadir + 'KnowledgeDatabase/GamepediaTxt/Entity',
    datadir + 'KnowledgeDatabase/GamepediaTxt/Items',
    datadir + 'KnowledgeDatabase/GamepediaTxt/Other'
]
minecraft_data = read_files_to_json(folder_paths, entities_list)


In [ ]:
import re
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Function to extract entities from ontology files
def parse_entities_from_file(file_path):
    entities = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Skip lines that don't contain entity definitions
            if line.startswith("<") or line.strip() == '':
                continue
            # Extract entity, ignoring hierarchy indentation
            entity = line.strip().split('\t')[-1]
            entities.append(entity)
    return entities

# Function to get the wordnet POS tag
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun

# Function to preprocess text
def preprocess_text(text, entity_list):
    lemmatizer = WordNetLemmatizer()
    tokenizer = RegexpTokenizer(r'\w+|[ENTITY][^/]+[/ENTITY]')  # Adjust the tokenizer to capture entity tags as single tokens
    tokens = tokenizer.tokenize(text)

    processed_tokens = []
    for token in tokens:
        if token.startswith('[ENTITY]'):
            processed_tokens.append(token)  # Keep the entity as is
        else:
            # Lowercase non-entity tokens
            token = token.lower()
            if token not in stopwords.words('english'):
                # Get POS tags for lemmatization
                pos = pos_tag([token])[0][1]
                wordnet_pos = get_wordnet_pos(pos)  # Convert the tag to wordnet format
                # Lemmatize the token
                lemmatized_token = lemmatizer.lemmatize(token, wordnet_pos)
                processed_tokens.append(lemmatized_token)
    return ' '.join(processed_tokens)

# Example usage
# entities_list = parse_entities_from_file('ontology.txt')  # Assuming 'ontology.txt' contains your ontology entities
sample_text = "To create a [ENTITY]Nether Portal[/ENTITY], you need [ENTITY]Obsidian[/ENTITY] which can be mined with a [ENTITY]Diamond Pickaxe[/ENTITY]."
preprocessed_sample = preprocess_text(sample_text, entities_list)
print(preprocessed_sample)

create nether portal need entity obsidian entity mine entity diamond pickaxe entity


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun

def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text.lower())
    tokens = [t for t in tokens if t not in stopwords.words('english')]
    pos_tags = pos_tag(tokens)  # Get POS tags
    tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tags]  # Lemmatize with POS tags
    return ' '.join(tokens)

with open('minecraft_data.json', 'r', encoding='utf-8') as f:
    minecraft_data = json.load(f)

preprocessed_data = {key: preprocess_text(value['description']) for key, value in minecraft_data.items()}

with open('preprocessed_minecraft_data.json', 'w', encoding='utf-8') as f:
    json.dump(preprocessed_data, f, indent=4)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
!pip install elasticsearch

In [ ]:
es_url = "https://134.209.74.243:9200/"
from elasticsearch import Elasticsearch

es = Elasticsearch(
    ['https://134.209.74.243:9200'],
    verify_certs=False,
    http_auth=('elastic', 'mmjIugA2iaP=y0q2*5=4')
)
# es = Elasticsearch(hosts = [ES_NODES])


/usr/local/lib/python3.10/dist-packages/elasticsearch/_sync/client/__init__.py:397: SecurityWarning: Connecting to 'https://134.209.74.243:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
<ipython-input-6-5c3c09fe588c>:4: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


In [ ]:
es.indices.create(index='minecraft')

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'minecraft'})

In [ ]:
for topic, description in preprocessed_data.items():
    print(f"Topic: {topic}")
    print(f"Description: {description[:50]}...")

    doc = {
        'topic': topic,
        'description': description
    }

    try:
        res = es.index(index='minecraft', id=topic, body=doc)
        print(f"Indexed {topic}")
    except Exception as e:
        print(f"Failed to index {topic}: {e}")

Topic: Cauldron
Description: cauldron block hold water cauldron mine use pickax...
Indexed Cauldron
Topic: Banner
Description: banner flag tall decorative block feature field hi...


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Indexed Banner
Topic: Andesite
Description: andesite type igneous rock polish andesite polish ...
Indexed Andesite
Topic: Block_of_Coal
Description: block coal also know coal block mineral block also...
Indexed Block_of_Coal
Topic: Block_of_Iron
Description: block iron also know iron block decorative mineral...


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Indexed Block_of_Iron
Topic: Cake
Description: cake food block eat player place cake cannot recol...
Indexed Cake
Topic: Chest
Description: chest block store item chest obtain craft break pr...
Indexed Chest
Topic: Cactus
Description: cactus plural cactus cactus plant occur naturally ...


...

Indexed Stick
Topic: Sword
Description: sword melee weapon monster spawn sword 8 5 9 5 loo...
Indexed Sword
Topic: Spider_Eye
Description: spider eye poisonous food brewing item spider cave...
Indexed Spider_Eye
Topic: Spawn_Egg
Description: spawn egg item may use spawn mobs directly allow p...


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Indexed Spawn_Egg
Topic: Shovel
Description: shovel tool use ease process collect dirt block zo...
Indexed Shovel
Topic: Shears
Description: shear tool shepherd villager sell shear 3 4 emeral...
Indexed Shears
Topic: Tools
Description: tool item use player hold perform action faster ef...


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Indexed Tools


In [ ]:
def search_minecraft(query):
    # Define the search request
    search_request = {
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["topic", "description"]
            }
        }
    }

    response = es.search(index="minecraft", body=search_request)

    hits = response['hits']['hits']
    results = []
    for hit in hits:
        topic = hit['_source']['topic']
        description = hit['_source']['description']
        results.append((topic, description))

    return results

query = "What is a Cauldron used for?"
results = search_minecraft(query)
for topic, description in results:
    print(f"Topic: {topic}")
    print(f"Description: {description}\n")

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Topic: Cauldron
Description: cauldron block hold water cauldron mine use pickaxe mine without pickaxe drop nothing cauldron destroy water inside lose cauldron craft iron ingot single empty cauldron generate witch hut fill cauldron water press use cauldron water bucket cauldron also chance fill water rain upon water cauldron use fill glass bottle turn water bottle wash dye leather armor remove top pattern layer banner use cauldron press use cauldron glass bottle leather armor banner cauldron extinguish mob fire include player fall use include extinguish mobs cause water level cauldron decrease one third use three time empty must refill additional us endermen fill cauldron take damage water cauldron cannot use fill empty bucket water bottle cannot use refill cauldron cauldron fill water bucket nether cauldron act power source redstone comparator cauldron behind possibly separate unpowered solid block comparator output signal strength proportional full cauldron 0 empty 1 one third full 2 

In [ ]:
import nltk

question = "What is the use of a redstone comparator in Minecraft?"

tokens = nltk.word_tokenize(question)

In [ ]:
pos_tags = nltk.pos_tag(tokens)

In [ ]:
keywords = [word for word, pos in pos_tags if pos in ('NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ')]

In [ ]:
index_name = 'minecraft'

def fetch_indexed_data(index_name):
    indexed_data = {}

    result = es.search(index=index_name, scroll='1m', size=1000)
    scroll_id = result['_scroll_id']
    while len(result['hits']['hits']):
        for hit in result['hits']['hits']:
            doc_id = hit['_id']
            doc_source = hit['_source']
            indexed_data[doc_id] = doc_source
        result = es.scroll(scroll_id=scroll_id, scroll='1m')
        scroll_id = result['_scroll_id']

    return indexed_data

indexed_data = fetch_indexed_data(index_name)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
keywords = [keyword.lower() for keyword in keywords]

# Check if any of the keywords match the keys in your indexed data
matching_entities = [entity for entity in indexed_data if any(keyword in entity.lower() for keyword in keywords)]
relevant_data = {entity: indexed_data[entity] for entity in matching_entities}

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

documents = [data['description'] for data in relevant_data.values()]

texts = [question] + documents

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()

document_ranking = [documents[i - 1] for i in cosine_similarities.argsort()[::-1] if i != 0]



In [ ]:
import re
from nltk.tokenize import sent_tokenize

# Function to extract keywords from a text
def extract_keywords(text):
    return set(re.findall(r'\b\w+\b', text.lower()))

# Get the keywords from the user's question
question_keywords = extract_keywords(question)

# Initialize an empty list to store sentences and their scores
candidate_answers = []

# Go through each document in the ranked list
for doc in document_ranking:
    # Tokenize the document into sentences
    sentences = sent_tokenize(doc)

    # Go through each sentence in the document
    for sentence in sentences:
        # Get the keywords in the sentence
        sentence_keywords = extract_keywords(sentence)

        # Compute a score based on keyword overlap
        score = len(question_keywords.intersection(sentence_keywords))

        # Append the sentence and its score to the list of candidates
        candidate_answers.append((sentence, score))

# Sort the candidate answers by their scores in descending order
candidate_answers.sort(key=lambda x: x[1], reverse=True)

# The top-ranked sentence is your answer (or you could return the top N sentences)
answer = candidate_answers[0][0]

# Print the answer
print(answer)

NameError: ignored

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Load the pretrained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

def get_answer(question, context):
    # Tokenize the input
    input_text = "[CLS] " + question + " [SEP] " + context + " [SEP]"
    input_ids = tokenizer.encode(input_text)
    token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]

    # Convert to PyTorch tensors and run the model
    input_ids = torch.tensor([input_ids])
    token_type_ids = torch.tensor([token_type_ids])
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=token_type_ids)

    # Get the most likely start and end positions for the answer
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits)

    # Get the answer text
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end+1]))

    return answer

# Assume the first document in the ranking is the most relevant
most_relevant_doc = document_ranking[0]

# Get the answer from the most relevant document
answer = get_answer(question, most_relevant_doc)
print(answer)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


pickaxe mine without pickaxe drop nothing
